# Here you test something

Cols meaning:
- Day - day of the month
- DKind - day kind
- EC - exercise count
- EI - exercise intensity - has six levels from 0 - didn't exercise to 5 - hard exercise (More info in a separate message)
- OD - outdoor
- LE - left-eye reading
- RN - running
- GM - gaming


(Separate message about EI)
Exercise intensity notation:
- 0 - you didn't exercise
- 1 - easy
- 2 - between easy and moderate
- 3 - moderate
- 4 - between moderate and hard
- 5 - hard

In [1]:
import numpy as np
import pandas as pd

from preparation.prepare_data import prepare_data
from constants import PATH, APPEND_PATH, PREVIOUS_DATA_FOLDER, DESIRED_MEAN_VALUE

rep_data, wide_use_data, feedback_data = prepare_data(PATH, DESIRED_MEAN_VALUE)
month, mean, std, df = wide_use_data
df

,Day,DKind,Math,CS,Eng,Total,EC,EI,OD,LE,RN,GM
0,1,relax,0,0,43,43,1,2,130,0,0,0
1,2,work,0,265,15,280,1,0,105,0,0,0
2,3,work,0,205,69,274,2,3,165,10,0,0
3,4,work,0,185,115,300,3,5,230,0,13,0
4,5,relax,0,0,19,19,4,3,160,10,0,0
5,6,work,0,260,66,326,5,4,210,0,11,0
6,7,work,0,195,114,309,6,4,140,10,0,0
7,8,relax,0,50,10,60,7,2,215,0,0,0
8,9,work,0,250,30,280,7,0,120,10,0,0
9,10,work,0,222,22,244,8,4,290,0,15,0


In [2]:
rep_data

([0, 69, 23, 92, 190, 10, '1, 5, 8, 12, 19, 20, 24, 25, 27, 28'],
 [['OD', 'LE', 'RN', 'GM'],
  22,
  [93.33, 2.25, 1.83, 46.92],
  [28, 13, 8, 8],
  {1: 0, 2: 5, 3: 6, 4: 7, 5: 4}])

# Working with 'hobby' data

Get the file content

In [3]:
import re
import pandas as pd

PATH = "test.txt"
# PATH = "C:/Users/San/Documents/inf/time monitoring/book, movies, and other stuff I completed by year/2023 - What I read, watched and played.txt"
# PATH = "C:/Users/San/Documents/inf/time monitoring/book, movies, and other stuff I completed by year/2022 - What I read, watched and played.txt"
# PATH = "C:/Users/San/Documents/inf/time monitoring/book, movies, and other stuff I completed by year/2021 - What I read, watched and played.txt"
# Read the text file with UTF-8 encoding
with open(PATH, "r", encoding="utf-8") as file:
    content = file.read()
print(content)

Books:
1. "The Republic of Thieves: Gentleman Bastards" by Scott Lynch | audiobook on January 5, 2023
2. "Hyperion" by Dan Simmons | audiobook on January 13, 2023
3. "Propaganda" by Edward Bernays | audiobook on Jan 17, 2023
4. "The Fall of Hyperion" by Dan Simmons | audiobook on Jan 23rd, 2023
5. "Джури-характерники" by Володимир Рутківський on January 23, 2023
6. "Zen in the Art of Writing" by Ray Bradbury | audiobook on January 25, 2023
7. "The Final Empire: Mistborn, Book 1" by Brandon Sanderson | audiobook on February 2, 2023
8. "The Well of Ascension: Mistborn, Book 2" by Brandon Sanderson | audiobook on February 4, 2023
9. "The Hero of Ages: Mistborn: Book 3" by Brandon Sanderson | audiobook on Feb 6, 2023
10. "Factotum" by Charles Bukowski | audiobook on February 9, 2023
11. "Ham on Rye" by Charles Bukowski | audiobook on February 12, 2023
12. "Warbreaker" by Brandon Sanderson on February 13, 2023
13. "Hollywood" by Charles Bukowski | audiobook on February 16, 2023
14. "Pulp" b

Get movie df

In [4]:
# Find the 'Movies:' section using the regular expression
movies_section = re.search(r"Movies:(.*?)(?=\n\n|$)", content, re.DOTALL).group(1).strip()
# Define a regular expression pattern for extracting movie information
movie_pattern = re.compile(r'\d+\. ([^\n]+) on ([A-Za-z]+) (\d{1,2})(?:st|nd|rd|th)?, (\d{4})')
# Extract movie information using the pattern
movie_data = movie_pattern.findall(movies_section)
cols = ["Name", "Month", "Day", "Year"]
# Create a pandas DataFrame for movies
movies_df = pd.DataFrame(movie_data, columns=cols)
# Shorten the month to the first three letters
movies_df["Month"] = movies_df["Month"].apply(lambda month: month[:3])
movies_df

,Name,Month,Day,Year
0,Romancing the Stone,Jan,6,2023
1,Star Trek: The Wrath of Khan,Jan,15,2023
2,Greyhound,Jan,16,2023
3,The Thing,Jan,19,2023
4,"Lock, Stock and Two Smoking Barrels",Jan,6,2022
5,Revolver,Jan,6,2022
6,Wrath of Man,Jan,8,2022


Get games df

In [5]:
# Find the 'Games:' section using the regular expression
games_section = re.search(r"Games:(.*?)$", content, re.DOTALL).group(1).strip()
# Define a regular expression pattern for extracting game information
game_pattern = re.compile(r'\d+\. ([^\n]+) on ([A-Za-z]+) (\d{1,2})(?:st|nd|rd|th)?, (\d{4}) \((\d+) hours\)')
# Extract game information using the pattern
game_data = game_pattern.findall(games_section)
# 'PTH' stands for 'Play Time in Hours'a
cols=["Name", "Month", "Day", "Year", "PTH"]
# Create a pandas DataFrame for games
games_df = pd.DataFrame(game_data, columns=cols)
# Shorten the month to the first three letters
games_df["Month"] = games_df["Month"].apply(lambda month: month[:3])
games_df

,Name,Month,Day,Year,PTH
0,Marvel's Guardians of the Galaxy,Jan,15,2023,25
1,Titanfall 2,Feb,19,2023,6
2,Xcom2 WotC,Feb,28,2022,50
3,Ghostrunner,Mar,15,2022,11
4,Star Wars Jedi Fallen Order,Apr,6,2022,22
5,Kingdom New Lands,Dec,29,2021,14


Get books df

In [6]:
# Find the 'Books:' section using string manipulation
books_section = re.search(r"Books:(.*?)(?=Movies:|Games:)", content, re.DOTALL).group(1).strip()
# Split the section into lines
book_lines = books_section.split('\n')
# Remove all data before the first double quote in each line
book_lines = [line.split('"', 1)[1] for line in book_lines]
book_lines[:3]

['The Republic of Thieves: Gentleman Bastards" by Scott Lynch | audiobook on January 5, 2023',
 'Hyperion" by Dan Simmons | audiobook on January 13, 2023',
 'Propaganda" by Edward Bernays | audiobook on Jan 17, 2023']

In [7]:
# Initialize variables to store book data
book_data = []
audiobook_marker = "| audiobook"
# Process each line in the book section
for line in book_lines:
    cur_book = {}
    # Audiobook entry
    if audiobook_marker in line:
        # Split the data on book data (book name and author)
        # and date (Month, day, and year)
        # print(line)
        name_author, date = line.split(audiobook_marker)
        name, author = name_author.split('" by ')
        # Use 'strip()' to remove trailing whitespaces
        cur_book["Name"] = name.strip()
        cur_book["Author"] = author.strip()
        cur_book["Audiobook"] = True
    # Regular book entry
    else:
        # Handle the case when there's ' on ' in book name
        # e.g., 'Essays on the Theory of Numbers'
        try:
            name_author_pair, date = line.split(" on ")
        except ValueError:
            line_list = line.split(" on ")
            name_author_pair = " on ".join(line_list[:-1])
            date = line_list[-1]
        name, author = name_author_pair.split('" by ')
        # Use 'strip()' to remove trailing whitespaces
        cur_book["Name"] = name.strip()
        cur_book["Author"] = author.strip()
        cur_book["Audiobook"] = False
    # Remove trailing whitespace before the string
    # Otherwise, regex won't work
    date = date.strip()
    # Extract date info using regex
    match = re.match(r'(?:on )?([A-Za-z]+) (\d{1,2})(?:st|nd|rd|th)?, (\d{4})', date)
    cur_book["Month"] = match.group(1)[:3]
    cur_book["Day"] = int(match.group(2))
    cur_book["Year"] = int(match.group(3))
    book_data.append(cur_book)

# Create a pandas DataFrame for books
books_df = pd.DataFrame(book_data, columns=["Name", "Author", "Audiobook", "Month", "Day", "Year"])
books_df.head(50)

,Name,Author,Audiobook,Month,Day,Year
0,The Republic of Thieves: Gentleman Bastards,Scott Lynch,True,Jan,5,2023
1,Hyperion,Dan Simmons,True,Jan,13,2023
2,Propaganda,Edward Bernays,True,Jan,17,2023
3,The Fall of Hyperion,Dan Simmons,True,Jan,23,2023
4,Джури-характерники,Володимир Рутківський,False,Jan,23,2023
5,Zen in the Art of Writing,Ray Bradbury,True,Jan,25,2023
6,"The Final Empire: Mistborn, Book 1",Brandon Sanderson,True,Feb,2,2023
7,"The Well of Ascension: Mistborn, Book 2",Brandon Sanderson,True,Feb,4,2023
8,The Hero of Ages: Mistborn: Book 3,Brandon Sanderson,True,Feb,6,2023
9,Factotum,Charles Bukowski,True,Feb,9,2023
